# 卡牌效果生成器

本 Notebook 演示如何使用 LangChain 构建卡牌效果生成系统。我们将使用 Pydantic 模型来定义和验证输出格式，使用 PromptTemplate 来构建提示词。

In [ ]:
# 导入必要的库
from typing import Dict, List, Any, Union, Optional
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator

## 1. 定义数据模型
首先定义卡牌效果相关的数据模型

In [ ]:
# 定义卡牌效果类型
class CardEffect(BaseModel):
    effect_type: str = Field(..., description="效果类型")
    target_type: str = Field(..., description="目标类型")
    value: Union[int, str] = Field(..., description="效果值")
    condition: Optional[str] = Field(None, description="触发条件")
    duration: Optional[int] = Field(None, description="持续回合数")

# 定义指令参数模型
class InstructionParameters(BaseModel):
    card_id: Optional[str] = Field(None, description="卡牌ID")
    target_position: Optional[str] = Field(None, description="目标位置")
    source: Optional[str] = Field(None, description="来源位置")
    animation_name: Optional[str] = Field(None, description="动画名称")
    target_id: Optional[str] = Field(None, description="目标ID")
    value: Optional[int] = Field(None, description="数值")
    type: Optional[str] = Field(None, description="类型")
    effect: Optional[CardEffect] = Field(None, description="卡牌效果")

## 2. 定义指令模型

In [ ]:
class Instruction(BaseModel):
    action: str = Field(..., description="指令类型")
    parameters: InstructionParameters = Field(..., description="指令参数")
    duration: float = Field(..., description="执行时长")
    sequence: int = Field(..., description="执行序号")

    @validator('action')
    def validate_action(cls, v):
        valid_actions = [
            'MOVE_CARD', 'PLAY_ANIMATION', 'UPDATE_HEALTH',
            'SHOW_MESSAGE', 'CREATE_CARD', 'APPLY_EFFECT'
        ]
        if v not in valid_actions:
            raise ValueError(f'动作必须是以下之一: {valid_actions}')
        return v

## 3. 创建命令生成器

In [ ]:
class CommandOutput(BaseModel):
    card_id: str = Field(..., description="卡牌ID")
    instructions: List[Instruction] = Field(..., description="指令列表")
    state_updates: Dict[str, Any] = Field(..., description="状态更新")

class CardCommandGenerator:
    def __init__(self):
        self.output_parser = PydanticOutputParser(pydantic_object=CommandOutput)
        
    def generate_command_template(self, action: str):
        # 实现命令模板生成逻辑
        pass
    
    def format_prompt(self, game_state: Dict, card_data: Dict, player_action: str):
        # 实现提示词格式化逻辑
        pass

## 4. 示例：生成卡牌效果

In [ ]:
# 创建示例游戏状态
game_state = {
    "player_health": 30,
    "opponent_health": 30,
    "player_field": [],
    "opponent_field": []
}

# 创建示例卡牌数据
card_data = {
    "id": "CARD_001",
    "name": "火球术",
    "type": "spell",
    "cost": 4,
    "effect": {
        "type": "damage",
        "value": 6
    }
}

# 创建命令生成器实例
generator = CardCommandGenerator()

# 生成命令
prompt = generator.format_prompt(game_state, card_data, "play_card")
print("生成的提示词：\n", prompt)